# Projeto 1 - Ciência dos Dados

Nome: Ana Carolina

Nome: Henrique Gabriel

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [3]:
import os

filename = 'BaseDeDadosProjeto1.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo BaseDeDadosProjeto1.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [4]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Avaliação
0,@robsonbrito204 @isadorabasile mmmmeeeeeeu dee...,3
1,@emvideogame @xramoses @playstation_br com ess...,5
2,@eleitor73 @windowsclubbr playstation é forte ...,1
3,@funimation_bra @ccxpoficial queria saber sobr...,5
4,@fuck_off_matt eles vão ter que se acostumar c...,1


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Avaliação
0,@facundini_ @izzynobre eles tem praticamente t...,2
1,@traiinbe eu acho que se você é gamer hardcore...,1
2,@monstrohormonal @diegolt85 @playstation_br pe...,5
3,normalmente quem prefere android prefere xbox\...,5
4,borderlands 3 será lançado para playstation 5 ...,5


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

`Tendo em vista o Projeto 1 da matéria de Ciência de Dados, proposto pela Professora Maria Kelly Venezuela, escolhemos os produtos Xbox e PlayStation, os quais são marcas de consoles desenvolvidos pela Microsoft e pela Sony, respectivamente. 
    Logo, consideramos como "não relevante" qualquer tipo de postagem que usasse o @ desses produtos apenas para promover algo próprio, como por exemplo um canal no YouTube, ou qualquer tipo de postagem que embora utilizasse dos nomes dos produtos, não se relacionava com o mesmo. 
    Assim consideramos como "relevante" qualquer tipo de postagem que trata-se de certa forma a opinião sobre os produtos em questão.`

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [6]:
#Célula com função para limpar o DF

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[,!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [7]:
#Limpando o DF, transforando letras maiusculas em minusculas e passando o texto para uma lista de palavras

todas_as_palavras_ps = []
todas_as_palavras_indi = []
todas_as_palavras_xbox = []
todas_as_palavras_irr = []

for i, texto in enumerate(train.Treinamento):
    
    texto = cleanup(str(texto).lower())
    
    if train.Avaliação[i] == 5:
        todas_as_palavras_irr += texto.split()
    
    elif train.Avaliação[i] == 1: 
        todas_as_palavras_ps += texto.split()
    
    elif train.Avaliação[i] == 2: 
        todas_as_palavras_indi += texto.split()
    
    else:
        todas_as_palavras_xbox += texto.split()

In [8]:
#LISTA COM PALAVRAS COMUNS QUE PODEM SER "LIMPADAS" DO DF 

palavras_desnecessarias = ["a", "o", "e", "da", "do", "em", "dos", "das", "que", "de"]

In [9]:
#limpando @'s dos usuarios e palavras desnecessarias

todas_as_palavras_irr_arroba = []

for num, palavra in enumerate(todas_as_palavras_irr):
    if ("@" not in palavra) and (palavra not in palavras_desnecessarias):
        todas_as_palavras_irr_arroba.append(palavra)

todas_as_palavras_irr_arroba[0:10]

['com',
 'esse',
 'reajuste',
 'serviços',
 'xbox',
 'dá',
 'mesmo',
 'total',
 'você',
 'assinava']

In [10]:
#limpando @'s dos usuarios e palavras desnecessarias

todas_as_palavras_ps_arroba = []

for num, palavra in enumerate(todas_as_palavras_ps):
    if ("@" not in palavra) and (palavra not in palavras_desnecessarias):
        todas_as_palavras_ps_arroba.append(palavra)

todas_as_palavras_ps_arroba[0:10]

['playstation',
 'é',
 'forte',
 'no',
 'mercado',
 'europeu',
 'asiático',
 'também',
 'tem',
 'uma']

In [11]:
#limpando @'s dos usuarios e palavras desnecessarias

todas_as_palavras_indi_arroba = []

for num, palavra in enumerate(todas_as_palavras_indi):
    if ("@" not in palavra) and (palavra not in palavras_desnecessarias):
        todas_as_palavras_indi_arroba.append(palavra)

todas_as_palavras_indi_arroba[0:10]

['conversando',
 'com',
 'um',
 'amigo',
 'cheguei',
 'conclusão',
 'na',
 'década',
 '90',
 'os']

In [12]:
#limpando @'s dos usuarios e palavras desnecessarias

todas_as_palavras_xbox_arroba = []

for num, palavra in enumerate(todas_as_palavras_xbox):
    if ("@" not in palavra) and (palavra not in palavras_desnecessarias):
        todas_as_palavras_xbox_arroba.append(palavra)

todas_as_palavras_xbox_arroba[0:10]

['mmmmeeeeeeu',
 'deeeeeeeeeeeeeeeuuuussss',
 'retardado',
 'joga',
 'playstation',
 'quer',
 'exigir',
 'uma',
 'gtgtgt',
 'apresentadora']

In [13]:
#Transformando lista em um Series

serie_palavras_irr = pd.Series(todas_as_palavras_irr_arroba)

serie_palavras_ps = pd.Series(todas_as_palavras_ps_arroba)

serie_palavras_indi = pd.Series(todas_as_palavras_indi_arroba)

serie_palavras_xbox = pd.Series(todas_as_palavras_xbox_arroba)

In [14]:
# Adquirindo a tabela de frequências relativas a partir de um dos Series criados acima

tabela_tweets_relativa_irr = serie_palavras_irr.value_counts(True)
tabela_tweets_relativa_irr

xbox                     0.040561
playstation              0.034496
para                     0.017058
rt                       0.012509
x                        0.012509
                           ...   
indiegame                0.000379
ae                       0.000379
tá                       0.000379
https//tco/tqdbm3swo6    0.000379
343                      0.000379
Length: 1162, dtype: float64

In [15]:
# Adquirindo a tabela de frequências relativas a partir de um dos Series criados acima

tabela_tweets_relativa_ps = serie_palavras_ps.value_counts(True)
tabela_tweets_relativa_ps

xbox           0.042032
playstation    0.029795
não            0.025273
é              0.024741
mais           0.017824
                 ...   
quintafeira    0.000266
ação           0.000266
strange        0.000266
salto          0.000266
andar          0.000266
Length: 1267, dtype: float64

In [16]:
# Adquirindo a tabela de frequências relativas a partir de um dos Series criados acima

tabela_tweets_relativa_indi = serie_palavras_indi.value_counts(True)
tabela_tweets_relativa_indi

xbox                     0.043510
playstation              0.037037
não                      0.017620
é                        0.017260
no                       0.015102
                           ...   
https//tco/kbt0pm7qqm    0.000360
farpas                   0.000360
drake                    0.000360
véio                     0.000360
amigos                   0.000360
Length: 1101, dtype: float64

In [17]:
# Adquirindo a tabela de frequências relativas a partir de um dos Series criados acima

tabela_tweets_relativa_xbox = serie_palavras_xbox.value_counts(True)
tabela_tweets_relativa_xbox

xbox               0.045388
playstation        0.030357
é                  0.029472
não                0.021515
mais               0.018273
                     ...   
unificada          0.000295
marcou             0.000295
lançamentoo        0.000295
playstationquem    0.000295
acessibilidade     0.000295
Length: 1162, dtype: float64

In [18]:
#Limpando o DF, transforando letras maiusculas em minusculas e passando o texto para uma lista de palavras que representaram o conjunto LÍNGUA PORTUGUESA

todas_as_palavras_test = []

for texto in test.Teste:
    texto = cleanup(str(texto).lower())
    todas_as_palavras_test += texto.split()

In [20]:
#Transformando lista em um Series e adquirindo a tabela de frequências relativas a partir de um mesmo Series.

portugues = todas_as_palavras_irr + todas_as_palavras_ps + todas_as_palavras_test + todas_as_palavras_indi + todas_as_palavras_xbox

#limpando @'s dos usuarios e palavras desnecessarias

portugues_arroba = []

for num, palavra in enumerate(portugues):
    if ("@" not in palavra) and (palavra not in palavras_desnecessarias):
        portugues_arroba.append(palavra)


serie_portugues = pd.Series(portugues_arroba)
tabela_portugues_relativa = serie_portugues.value_counts(True)
tabela_portugues_relativa

xbox            0.043945
playstation     0.032678
é               0.020344
não             0.017354
mais            0.013189
                  ...   
sinceramente    0.000053
bdzt            0.000053
admite          0.000053
vinculada       0.000053
desempenho      0.000053
Length: 4171, dtype: float64

In [21]:
#Dividindo entre Relevante ou Não Relevante de acordo com nossos critérios e nossas considerações

test.loc[(test.Avaliação == 5),"Rel_Or_Not"] = 'Not relevant'
test.loc[(test.Avaliação == 1),"Rel_Or_Not"] = 'PlayStation'
test.loc[(test.Avaliação == 2),'Rel_Or_Not'] = 'Indifferent'
test.loc[(test.Avaliação == 3),'Rel_Or_Not'] = 'Xbox'

In [150]:
# Calculando as probabilidades relativas de cada categoria 

# prob_rel_or_not = test.Avaliação.value_counts(True, sort=False)

# probPS = prob_rel_or_not[1]
# probINDI = prob_rel_or_not[2]
# probXBOX = prob_rel_or_not[3]
# probIRRE = prob_rel_or_not[5]

# probREL = probINDI + probPS + probXBOX

probPS = 0.25
probINDI = 0.25
probXBOX = 0.25
probIRRE = 0.25




print("probPS:",probPS)
print("probINDI:",probINDI)
print("probXBOX:",probXBOX)
print("probIRRE:",probIRRE)

probPS: 0.25
probINDI: 0.25
probXBOX: 0.25
probIRRE: 0.25


In [151]:
for n, tweet in enumerate(test.Teste):

    prob_tweet = 0

    tweet = cleanup(str(tweet).lower())
    tweet = tweet.split()

    for i, word in enumerate(tweet):
        if (word in todas_as_palavras_irr_arroba) or (word in todas_as_palavras_ps_arroba) or (word in todas_as_palavras_indi_arroba) or (word in todas_as_palavras_xbox_arroba):
            if prob_tweet == 0:
                prob_tweet = (tabela_portugues_relativa[word] + 1)
            else:
                prob_tweet = prob_tweet * (tabela_portugues_relativa[word] + 1)

    test.loc[n,'ProbTweet'] = prob_tweet / (i + 1)

In [152]:
#CALCULANDO A PROBABILIDADE DE UM TWEET DADO QUE É IRRELEVANTE (USANDO SUAVISAÇÃO DE LA PLACE)

for n, tweet in enumerate(test.Teste):

    prob_tweet = 0

    tweet = cleanup(str(tweet).lower())
    tweet = tweet.split()

    for i, word in enumerate(tweet):
        if word in todas_as_palavras_irr_arroba:
            if prob_tweet == 0:
                prob_tweet = (tabela_tweets_relativa_irr[word] + 1)
            else:
                prob_tweet = prob_tweet * (tabela_tweets_relativa_irr[word] + 1)
        else:
            if prob_tweet == 0:
                prob_tweet = 1
            else:
                prob_tweet = prob_tweet
        
    test.loc[n,'ProbTweet_IRR'] = prob_tweet / (i + 1)

In [153]:
#CALCULANDO A PROBABILIDADE DE UM TWEET DADO QUE PLAYSTATION É A CATEGORIA (USANDO SUAVISAÇÃO DE LA PLACE)

for n, tweet in enumerate(test.Teste):

    prob_tweet = 0

    tweet = cleanup(str(tweet).lower())
    tweet = tweet.split()

    for i, word in enumerate(tweet):
        if word in todas_as_palavras_ps_arroba:
            if prob_tweet == 0:
                prob_tweet = (tabela_tweets_relativa_ps[word] + 1)
            else:
                prob_tweet = prob_tweet * (tabela_tweets_relativa_ps[word] + 1)
        else:
            if prob_tweet == 0:
                prob_tweet = 1
            else:
                prob_tweet = prob_tweet
        
    test.loc[n,'ProbTweet_PS'] = prob_tweet / (i + 1)

In [154]:
#CALCULANDO A PROBABILIDADE DE UM TWEET DADO QUE É INDIFERENTE (USANDO SUAVISAÇÃO DE LA PLACE)

for n, tweet in enumerate(test.Teste):

    prob_tweet = 0

    tweet = cleanup(str(tweet).lower())
    tweet = tweet.split()

    for i, word in enumerate(tweet):
        if word in todas_as_palavras_indi_arroba:
            if prob_tweet == 0:
                prob_tweet = (tabela_tweets_relativa_indi[word] + 1)
            else:
                prob_tweet = prob_tweet * (tabela_tweets_relativa_indi[word] + 1)
        else:
            if prob_tweet == 0:
                prob_tweet = 1
            else:
                prob_tweet = prob_tweet
        
    test.loc[n,'ProbTweet_INDI'] = prob_tweet / (i + 1)

In [155]:
#CALCULANDO A PROBABILIDADE DE UM TWEET DADO QUE XBOX É A CATEGORIA (USANDO SUAVISAÇÃO DE LA PLACE)

for n, tweet in enumerate(test.Teste):

    prob_tweet = 0

    tweet = cleanup(str(tweet).lower())
    tweet = tweet.split()

    for i, word in enumerate(tweet):
        if word in todas_as_palavras_xbox_arroba:
            if prob_tweet == 0:
                prob_tweet = (tabela_tweets_relativa_xbox[word] + 1)
            else:
                prob_tweet = prob_tweet * (tabela_tweets_relativa_xbox[word] + 1)
        else:
            if prob_tweet == 0:
                prob_tweet = 1
            else:
                prob_tweet = prob_tweet
        
    test.loc[n,'ProbTweet_XBOX'] = prob_tweet / (i + 1)

In [156]:
test.head()

,Teste,Avaliação,Rel_Or_Not,ProbTweet,ProbTweet_IRR,ProbTweet_PS,ProbTweet_INDI,ProbTweet_XBOX,ClassByCategoria
0,@facundini_ @izzynobre eles tem praticamente t...,2,Indifferent,0.025152,0.023014,0.026097,0.025154,0.025801,Xbox
1,@traiinbe eu acho que se você é gamer hardcore...,1,PlayStation,0.032916,0.031048,0.033468,0.032848,0.033593,Not relevant
2,@monstrohormonal @diegolt85 @playstation_br pe...,5,Not relevant,0.038366,0.038117,0.038307,0.038202,0.038516,Not relevant
3,normalmente quem prefere android prefere xbox\...,5,Not relevant,0.090444,0.090216,0.089995,0.090635,0.090449,Not relevant
4,borderlands 3 será lançado para playstation 5 ...,5,Not relevant,0.092572,0.094496,0.090932,0.094005,0.091627,Not relevant


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [157]:
for n, tweet in enumerate(test.Teste):
    
    prob_irr_dado_tweet = test.loc[n, 'ProbTweet_IRR'] * probIRRE / test.loc[n, 'ProbTweet']
    prob_ps_dado_tweet = test.loc[n, 'ProbTweet_PS'] * probPS / test.loc[n, 'ProbTweet']
    prob_indi_dado_tweet = test.loc[n, 'ProbTweet_INDI'] * probINDI / test.loc[n, 'ProbTweet']
    prob_xbox_dado_tweet = test.loc[n, 'ProbTweet_XBOX'] * probXBOX / test.loc[n, 'ProbTweet']

    lista_para_ver_maior_prob = [prob_irr_dado_tweet, prob_ps_dado_tweet, prob_indi_dado_tweet, prob_xbox_dado_tweet]

    if max(lista_para_ver_maior_prob) == prob_irr_dado_tweet:
        test.loc[n,"ClassByCategoria"] = "Not relevant"
    
    elif max(lista_para_ver_maior_prob) == prob_ps_dado_tweet:
        test.loc[n,"ClassByCategoria"] = "PlayStation"
    
    elif max(lista_para_ver_maior_prob) == prob_indi_dado_tweet:
        test.loc[n,"ClassByCategoria"] = "Indifferent"
    
    else:
        test.loc[n,"ClassByCategoria"] = "Xbox"

In [158]:
pd.crosstab(test.Rel_Or_Not, test.ClassByCategoria)

ClassByCategoria,Indifferent,Not relevant,PlayStation,Xbox
Rel_Or_Not,,,,
Indifferent,12,2,7,13
Not relevant,18,42,11,17
PlayStation,15,1,17,41
Xbox,28,2,16,35


In [159]:
pd.crosstab(test.Rel_Or_Not, test.ClassByCategoria, normalize = "index")*100

ClassByCategoria,Indifferent,Not relevant,PlayStation,Xbox
Rel_Or_Not,,,,
Indifferent,35.294118,5.882353,20.588235,38.235294
Not relevant,20.454545,47.727273,12.500000,19.318182
PlayStation,20.270270,1.351351,22.972973,55.405405
Xbox,34.567901,2.469136,19.753086,43.209877


___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**